<a href="https://colab.research.google.com/github/AI-FREE-Team/Machine-Learning-Basic/blob/master/Materials/%E7%BC%BA%E5%A4%B1%E5%80%BC_Missing_Value.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![Logo](https://raw.githubusercontent.com/AI-FREE-Team/Machine-Learning-Basic/main/README_imgs/aifreeteam.png) 
<center>Welcome to the course《Python: from business analytics to Artificial Intelligence》by AI . FREE Team.</center>
<center>歡迎大家來到 AI . FREE Team 《Python 從商業分析到人工智慧》的第二堂課 - 機器學習(ML)基礎教學。 </center>
<br/>

<center>作者: Tom Wu (<a href="https://github.com/YenLinWu">Github</a>) &nbsp&nbsp&nbsp 日期: June 20, 2021 </center>

# 前言 
本篇文章將介紹如何處理資料集中的<font color="#00dd00">**缺失值( Missing Value )**</font>，它在 Pandas 套件的物件中表示為 NaN 或 None 。資料集中存有缺失值的原因，常因資料的收集與使用場景有關，例如 : 面試時，有經驗的應徵者在填寫資料時，通常不願意直接表明期望薪資等。然而，對於某些機器學習的模型，是不允許資料集中有缺失值的存在，因此，在處理缺失值時，通常我們會先試著推測產生缺失值的原因，及是否與其他變數存有關聯性，再決定對缺失值採用刪除或填補的方式。讀者們在閱讀本篇文章後，將具備下列能力：
  1. 檢視資料集中缺失值的分佈情況；   
  2. 刪除資料集中具有缺失值的欄或列；  
  3. 填補資料集中的缺失值。


# 匯入套件

In [ ]:
import sys
import numpy as np
import pandas as pd

print( 'Python Version:', sys.version[0:7] ) 
print( 'Numpy Version:', np.__version__ )
print( 'Pandas Version:', pd.__version__ )

Python Version: 3.7.10 
Numpy Version: 1.19.5
Pandas Version: 1.1.5


# **檢視**缺失值 : [`pandas.DataFrame.isnull()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.isnull.html)    

使用 `DataFrame.isnull()` 函數可得知 DataFrame 物件中每個欄位的缺失值分佈，回傳值為 True 表示缺失值。然而，當資料筆數非常龐大時，我們並無法只透過這方法來獲得缺失值的狀況，因此，我們可透過 `DataFrame.isnull().sum()` 來計算資料集中每個欄位缺失值的總數，進一步得知各欄位的缺失值占比，而瞭解該資料集的品質：

In [ ]:
# Build the DataFrame
df = pd.DataFrame( { 'column_A':['A1','A2','A3','A4'],
                     'column_B':[np.nan,'B2',None,'B4'], 
                     'column_C':['C1',np.nan,'C3','C4'] } )

print( f'df : \n{df}\n' )
print( f'df 的缺失值分佈：\n{df.isnull()}\n' )
print( f'df 每個欄位缺失值的總數：\n{df.isnull( ).sum()}\n' )

df : 
  column_A column_B column_C
0       A1      NaN       C1
1       A2       B2      NaN
2       A3     None       C3
3       A4       B4       C4

df 的缺失值分佈：
   column_A  column_B  column_C
0     False      True     False
1     False     False      True
2     False      True     False
3     False     False     False

df 每個欄位缺失值的總數：
column_A    0
column_B    2
column_C    1
dtype: int64



定義一個函數，用以計算資料集的每個欄位缺失值的總數與比例 :

In [ ]:
def Missing_Counts( Data, NoMissing=True ) : 
    missing = Data.isnull().sum()  
    
    if NoMissing==False :
        missing = missing[ missing>0 ]
        
    missing.sort_values( ascending=False, inplace=True )  
    Missing_Count = pd.DataFrame( { 'Column Name':missing.index, 'Missing Count':missing.values } ) 
    Missing_Count[ 'Percentage(%)' ] = Missing_Count['Missing Count'].apply( lambda x: '{:.2%}'.format(x/Data.shape[0] ))
    return  Missing_Count

In [ ]:
Missing_Counts(df)

,Column Name,Missing Count,Percentage(%)
0,column_B,2,50.00%
1,column_C,1,25.00%
2,column_A,0,0.00%


# **刪除**缺失值 : [`pandas.DataFrame.dropna()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html)     

使用 `DataFrame.dropna( axis=0, how=any', thresh, subset )` 可刪除具有缺失值的欄或列，參數說明如下：  
  * axis = 指定刪除欄或列資料，0 為刪除列、1 為刪除欄，預設值為 0 ；  
  * how = 刪除具有 NaN 或 None 資料的方式，有下列兩種：  
  &emsp; 'all' : 刪除整欄或整列資料皆為 NaN 或 None 的資料；  
  &emsp; 'any' ( 預設值 ) : 刪除欄或列具有任何一個 NaN 或 None 的資料；
  * thresh = 保留至少有 n 個非 NaN 或 None 的欄或列資料。    

透過下面範例來說明 `DataFrame.dropna()` 函數的用法及效果，首先，建立 df 記載姓名、居住城市、身高及體重的資訊，欄位名稱如下：  
  * df：姓名( Name )、居住城市( City )、身高( Height )、體重( Weight )。 

In [ ]:
# df 紀錄姓名、居住城市、身高、體重 
df = pd.DataFrame( { 'Name':['Andy','Ken','Ryan',np.nan,'Tom'],
                     'City':['Taipei',None,'Chiayi',np.nan,'Hualien'],
                     'Height':[175,179,np.nan,np.nan,np.nan],
                     'Weight':[78,68,76,np.nan,np.nan] } )

print( f'df : \n{df}\n' )

df : 
   Name     City  Height  Weight
0  Andy   Taipei   175.0    78.0
1   Ken     None   179.0    68.0
2  Ryan   Chiayi     NaN    76.0
3   NaN      NaN     NaN     NaN
4   Tom  Hualien     NaN     NaN



`DataFrame.dropna()` 函數中 axis 參數的功用：

In [ ]:
# 刪除具有 NaN 的列
D1 = df.dropna( )

# 刪除具有 NaN 的欄
D2 = df.dropna( axis=1 )

print( f'df : \n{df}\n' )
print( f'D1 : 刪除具有 NaN 的列 \n{D1}\n' )
print( f'D2 : 刪除具有 NaN 的欄 \n{D2}\n' )

df : 
   Name     City  Height  Weight
0  Andy   Taipei   175.0    78.0
1   Ken     None   179.0    68.0
2  Ryan   Chiayi     NaN    76.0
3   NaN      NaN     NaN     NaN
4   Tom  Hualien     NaN     NaN

D1 : 刪除具有 NaN 的列 
   Name    City  Height  Weight
0  Andy  Taipei   175.0    78.0

D2 : 刪除具有 NaN 的欄 
Empty DataFrame
Columns: []
Index: [0, 1, 2, 3, 4]



`DataFrame.dropna()` 函數中 how 參數的用法：

In [ ]:
# 刪除全為 NaN 的列
D3 = df.dropna( how='all' )

# 刪除有 NaN 的列
D4 = df.dropna( how='any' )

print( f'df : \n{df}\n' )
print( f'D3 : 刪除全為 NaN 的列資料 \n{D3}\n' )
print( f'D4 : 刪除有 NaN 的列資料 \n{D4}\n' )

df : 
   Name     City  Height  Weight
0  Andy   Taipei   175.0    78.0
1   Ken     None   179.0    68.0
2  Ryan   Chiayi     NaN    76.0
3   NaN      NaN     NaN     NaN
4   Tom  Hualien     NaN     NaN

D3 : 刪除全為 NaN 的列資料 
   Name     City  Height  Weight
0  Andy   Taipei   175.0    78.0
1   Ken     None   179.0    68.0
2  Ryan   Chiayi     NaN    76.0
4   Tom  Hualien     NaN     NaN

D4 : 刪除有 NaN 的列資料 
   Name    City  Height  Weight
0  Andy  Taipei   175.0    78.0



`DataFrame.dropna()` 函數中 thresh 參數的用法：

In [ ]:
# 保留至少有 3 個非 NaN 的列資料
D5 = df.dropna( thresh=3 )

# 保留至少有 3 個非 NaN 的欄位
D6 = df.dropna( axis=1, thresh=3 )

print( f'df : \n{df}\n' )
print( f'D5 : 保留至少有 3 個非 NaN 的列資料 \n{D5}\n' )
print( f'D6 : 保留至少有 3 個非 NaN 的欄位 \n{D6}\n' )

df : 
   Name     City  Height  Weight
0  Andy   Taipei   175.0    78.0
1   Ken     None   179.0    68.0
2  Ryan   Chiayi     NaN    76.0
3   NaN      NaN     NaN     NaN
4   Tom  Hualien     NaN     NaN

D5 : 保留至少有 3 個非 NaN 的列資料 
   Name    City  Height  Weight
0  Andy  Taipei   175.0    78.0
1   Ken    None   179.0    68.0
2  Ryan  Chiayi     NaN    76.0

D6 : 保留至少有 3 個非 NaN 的欄位 
   Name     City  Weight
0  Andy   Taipei    78.0
1   Ken     None    68.0
2  Ryan   Chiayi    76.0
3   NaN      NaN     NaN
4   Tom  Hualien     NaN



# **填補**缺失值 : [`pandas.DataFrame.fillna()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.fillna.html)  

填補為處理資料缺失值的另一個方法，對於數值型欄位缺失值的填補，我們常用該欄位的平均值( Mean )、中位數( Median )、眾數( Mode )來補值，甚至，也可透過機器學習演算法，例如 : 線性迴歸模型，利用預測值來填補缺失值。需留意一點的是，應避免填補前後的資料分佈有明顯的差異！    
使用 `DataFrame.fillna( axis= , value= , method= )` 可填補具有缺失值的欄或列，參數說明如下：  
  * axis = 依欄或列填補 NaN 或 None ，0 為刪除列、1 為刪除欄，預設值為 0 ；  
  * value = 用來填補 NaN 或 None 的資料 ；
  * method = 填補 NaN 或 None 的方式有下列兩種：  
  &emsp; 'backfill' / 'bfill' : 使用 NaN 或 None 的下一個非缺失值填補；   
  &emsp; 'pad' / 'ffill' : 使用 NaN 或 None 的上一個非缺失值填補。  

下面範例說明 `DataFrame.fillna()` 函數的用法及效果，建立 df 記載各縣市的降雨機率，欄位名稱如下： 
  * df：縣市( City )、地區( Area )、降雨率( Probability ) 。

In [ ]:
# df 紀錄縣市、地區、氣溫 
df = pd.DataFrame( { 'City':['基隆','台北','桃園','台中','高雄','屏東'],
                     'Area':['北部',None,'北部','中部','南部',np.nan],
                     'Probability':[np.nan,5,15,35,np.nan,55] } )

print( f'df : \n{df}\n' )

df : 
  City  Area  Probability
0   基隆    北部          NaN
1   台北  None          5.0
2   桃園    北部         15.0
3   台中    中部         35.0
4   高雄    南部          NaN
5   屏東   NaN         55.0



In [ ]:
# 使用 'Missing' 字串填補缺失值
F1 = df.fillna( value='Missing' )

# 使用 NaN 的下一個非 NaN 值補缺失值
F2 = df.fillna( method = 'bfill' )

# 使用 NaN 的上一個非 NaN 值補缺失值
F3 = df.fillna( method = 'ffill' )

print( f'df : \n{df}\n' )
print( f'F1 : 使用 Missing 字串填補 \n{F1}\n' )
print( f'F2 : 使用 NaN 或 None 的下一個非缺失值填補 \n{F2}\n' )
print( f'F3 : 使用 NaN 或 None 的上一個非缺失值填補 \n{F3}\n' )

df : 
  City  Area  Probability
0   基隆    北部          NaN
1   台北  None          5.0
2   桃園    北部         15.0
3   台中    中部         35.0
4   高雄    南部          NaN
5   屏東   NaN         55.0

F1 : 使用 Missing 字串填補 
  City     Area Probability
0   基隆       北部     Missing
1   台北  Missing           5
2   桃園       北部          15
3   台中       中部          35
4   高雄       南部     Missing
5   屏東  Missing          55

F2 : 使用 NaN 或 None 的下一個非缺失值填補 
  City Area  Probability
0   基隆   北部          5.0
1   台北   北部          5.0
2   桃園   北部         15.0
3   台中   中部         35.0
4   高雄   南部         55.0
5   屏東  NaN         55.0

F3 : 使用 NaN 或 None 的上一個非缺失值填補 
  City Area  Probability
0   基隆   北部          NaN
1   台北   北部          5.0
2   桃園   北部         15.0
3   台中   中部         35.0
4   高雄   南部         35.0
5   屏東   南部         55.0



下列分別使用 Probability 欄位的平均值 27.5 及中位數 25 來填補該欄位的缺失值：

In [ ]:
mean = df['Probability'].mean( )   # 平均值
F4 = df.fillna( value={ 'Probability':mean } )

median = df['Probability'].median( )   # 中位數
F5 = df.fillna( value={ 'Probability':median } )

print( f'df : \n{df}\n' )
print( f'F4 : 使用 Probability 欄位的平均值 {mean} 填補 Probability 的缺失值 \n{F4}\n' )
print( f'F5 : 使用 Probability 欄位的中位數 {median} 填補 Probability 的缺失值 \n{F5}\n' )

df : 
  City  Area  Probability
0   基隆    北部          NaN
1   台北  None          5.0
2   桃園    北部         15.0
3   台中    中部         35.0
4   高雄    南部          NaN
5   屏東   NaN         55.0

F4 : 使用 Probability 欄位的平均值 27.5 填補 Probability 的缺失值 
  City  Area  Probability
0   基隆    北部         27.5
1   台北  None          5.0
2   桃園    北部         15.0
3   台中    中部         35.0
4   高雄    南部         27.5
5   屏東   NaN         55.0

F5 : 使用 Probability 欄位的中位數 25.0 填補 Probability 的缺失值 
  City  Area  Probability
0   基隆    北部         25.0
1   台北  None          5.0
2   桃園    北部         15.0
3   台中    中部         35.0
4   高雄    南部         25.0
5   屏東   NaN         55.0



# 結語 
本文介紹了二種處理缺失值的方法：刪除及填補。處理缺失值時，除需先瞭解資料集的欄位定義與內容之外，專業領域的知識及經驗更是不可或缺。在採取刪除的方法時，我們可先計算缺失值的佔比，衡量刪除缺失值的合理性與妥適性，另一方面，在採用填補的方法時，常用該欄位的統計值作為填補值，例如：平均值、中位數或眾數等，甚至也可填補具特定含意或不可能出現的值，在此需注意的一個大原則，即避免填補前後該欄位的資料分佈差異過大。   
針對本文的內容，若讀者們有發現任何的錯誤或疑問，非常歡迎您[來信( yenlinwu1112@gmail.com )]( mailto:yenlinwu1112@gmail.com )給予建議及討論，讓我們一同來學習成長！

# 返回 [課程大綱](https://github.com/AI-FREE-Team/Machine-Learning-Basic)